# Create Unstructured Amazon Bedrock Knowledge Base for Octank Financial / 为 Octank Financial 创建非结构化 Amazon Bedrock 知识库


This notebook demonstrates how to create and configure an Amazon Bedrock Knowledge Base for unstructured data.

本笔记本演示如何为非结构化数据创建和配置 Amazon Bedrock 知识库。

The Knowledge Base integrates Amazon S3 as the data source for the Octank Financial 10K document and uses Amazon OpenSearch Serverless as the vector store. It enables RAG by powering queries over unstructured financial and business content.

该知识库集成 Amazon S3 作为 Octank Financial 10K 文档的数据源，并使用 Amazon OpenSearch Serverless 作为向量存储。它通过对非结构化财务和业务内容进行查询来启用 RAG。

This unstructured knowledge base will be used in conjunction with the structured knowledge base to create agentic RAG using Strands Agents.

此非结构化知识库将与结构化知识库结合使用，通过 Strands Agents 创建智能体 RAG。


## Setup and prerequisites / 设置和先决条件

### Prerequisites / 先决条件
* Python 3.10+
* AWS account / AWS 账户
* Amazon Bedrock foundation models enabled / 已启用 Amazon Bedrock 基础模型
* IAM role with permissions to create Amazon Bedrock Knowledge Base, Amazon S3 bucket, Amazon OpenSearch Serverless / 具有创建 Amazon Bedrock 知识库、Amazon S3 存储桶、Amazon OpenSearch Serverless 权限的 IAM 角色

Let's now install the requirement packages and define the needed clients to create our Amazon Bedrock Knowledge Base:

现在让我们安装所需的包并定义创建 Amazon Bedrock 知识库所需的客户端：


In [1]:
import os
import json
import time
import uuid
import boto3
import logging
import requests
import botocore
from datetime import datetime


In [2]:
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = os.getenv("AWS_REGION")
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region)

print(f"AWS region: {region}")


AWS region: us-east-1


In [3]:
# Generate unique suffix for resource names
current_time = time.time()
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-4:]
suffix = f"{timestamp_str}"

print(f"Suffix: {suffix}")


Suffix: 1938


## Step 1: Download Amazon Bedrock Knowledge Bases helper / 步骤 1：下载 Amazon Bedrock 知识库助手
To expedite the knowledge base configuration and creation we will be downloading the knowledge base utility file. This contains a helper to generate knowledge bases abstracting the multiple API calls that need to be used.

为了加快知识库配置和创建，我们将下载知识库实用程序文件。它包含一个助手程序，用于生成知识库，抽象化需要使用的多个 API 调用。


In [4]:
from utils.knowledge_base import BedrockKnowledgeBase

## Step 2: Create Amazon Bedrock Knowledge Base for Unstructured Data / 步骤 2：为非结构化数据创建 Amazon Bedrock 知识库
In this section we will configure the Amazon Bedrock Knowledge Base containing the Octank Financial 10K document. We will be using Amazon OpenSearch Serverless Service as the underlying vector store and Amazon S3 as the data source containing the PDF file.

在本部分中，我们将配置包含 Octank Financial 10K 文档的 Amazon Bedrock 知识库。我们将使用 Amazon OpenSearch Serverless 服务作为底层向量存储，使用 Amazon S3 作为包含 PDF 文件的数据源。


In [5]:
knowledge_base_name = f"octank-financial-unstructured-kb-{suffix}"
knowledge_base_description = "Octank Financial Unstructured Knowledge Base containing 10K financial document for business strategy and company information queries."
foundation_model = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

For this notebook, we'll create a Knowledge Base with an Amazon S3 data source containing the Octank Financial 10K PDF.


In [6]:
data_bucket_name = f'octank-financial-unstructured-{suffix}-bucket'
data_sources = [{"type": "S3", "bucket_name": data_bucket_name}]


### Create the Amazon S3 bucket and upload the Octank Financial 10K document / 创建 Amazon S3 存储桶并上传 Octank Financial 10K 文档
We'll create an S3 bucket and upload the Octank Financial 10K PDF document that will serve as our unstructured data source.

我们将创建一个 S3 存储桶并上传 Octank Financial 10K PDF 文档，作为我们的非结构化数据源。


In [7]:
def create_s3_bucket(bucket_name, region=None):
    s3 = boto3.client('s3', region_name=region)

    try:
        if region is None or region == 'us-east-1':
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Bucket '{bucket_name}' created successfully.")
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
            print(f"Bucket '{bucket_name}' already exists and is owned by you.")
        else:
            print(f"Failed to create bucket: {e.response['Error']['Message']}")

create_s3_bucket(data_bucket_name, region)


Bucket 'octank-financial-unstructured-1938-bucket' created successfully.


### Create the Unstructured Knowledge Base / 创建非结构化知识库
We are now going to create the Knowledge Base using the abstraction located in the helper function we previously downloaded.

现在我们将使用之前下载的助手函数中的抽象来创建知识库。


In [10]:
unstructured_knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy="FIXED_SIZE", 
    suffix=f'{suffix}-u' 
)


[2025-09-05 05:19:50,268] p262804 {credentials.py:1138} INFO - Found credentials from IAM Role: CodeEditorV2-CodeEditorInstanceBootstrapRole-etk4tqcPN52E
[2025-09-05 05:19:50,622] p262804 {credentials.py:1138} INFO - Found credentials from IAM Role: CodeEditorV2-CodeEditorInstanceBootstrapRole-etk4tqcPN52E


Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['octank-financial-unstructured-1938-bucket']
buckets_to_check:  ['octank-financial-unstructured-1938-bucket']
Bucket octank-financial-unstructured-1938-bucket already exists - retrieving it!
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_1938-u) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '317',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Fri, 05 Sep 2025 05:19:51 '
                                                 'GMT',
                                         'x-amzn-requestid': '15420816-f755-4498-93b6-26ea381c8b7d'},
           

[2025-09-05 05:21:22,896] p262804 {base.py:258} INFO - PUT https://u6v807ii1kq2m0rgowt9.us-east-1.aoss.amazonaws.com:443/bedrock-sample-rag-index-1938-u [status:200 request:0.683s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-1938-u',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 9, 5, 5, 22, 23, 124653, tzinfo=tzlocal()),
  'description': 'Octank Financial Unstructured Knowledge Base containing 10K '
                 'financial document for business strategy and company '
                 'information queries.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:983760593521:knowledge-base/P26LUAGL1B',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'P26LUAGL1B',
  'name': 'octank-financial-unstructured-kb-1938',
  'roleArn': 'arn:aws:iam::98376

### Start upload and ingestion job / 开始上传和数据摄取作业
Once the KB and data source are created, we can start the ingestion job for the data source. During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database (OpenSearch Serverless).

一旦创建了知识库和数据源，我们就可以为数据源启动数据摄取作业。在数据摄取作业期间，知识库将获取数据源中的文档，预处理以提取文本，根据提供的分块大小进行分块，为每个分块创建嵌入，然后将其写入向量数据库（OpenSearch Serverless）。


In [ ]:
# Upload to S3 bucket

def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            print(f"Uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload, bucket_name, file)

# Upload the Octank Financial 10K document
upload_directory("./sample_unstructured_data", data_bucket_name)

In [11]:
# Ensure that the kb is available
time.sleep(30)
# Sync knowledge base
unstructured_knowledge_base.start_ingestion_job()
# Keep the kb_id for invocation later in the invoke request
unstructured_kb_id = unstructured_knowledge_base.get_knowledge_base_id()
print(f"Unstructured Knowledge Base ID: {unstructured_kb_id}")


job 1 started successfully

{ 'dataSourceId': 'ZAEDV1GF68',
  'ingestionJobId': 'JQDRRZDR52',
  'knowledgeBaseId': 'P26LUAGL1B',
  'startedAt': datetime.datetime(2025, 9, 5, 5, 23, 17, 620374, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 1},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 9, 5, 5, 24, 3, 356622, tzinfo=tzlocal())}
'P26LUAGL1B'............................
Unstructured Knowledge Base ID: P26LUAGL1B


### Test the Unstructured Knowledge Base / 测试非结构化知识库
We can now test the Knowledge Base to verify the Octank Financial 10K document has been ingested properly.

现在我们可以测试知识库以验证 Octank Financial 10K 文档是否已正确摄取。


In [12]:
query = "What is Octank Financial's primary business strategy?"

In [13]:
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': unstructured_kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults": 5
                } 
            }
        }
    }
)

print("Response:")
print(response['output']['text'], end='\\n'*2)


Response:
Octank Financial's primary business strategy is to provide a wide range of financial services to individuals, businesses, and institutions. Their core services include investment banking, wealth management, asset management, corporate finance, and private equity. They aim to help clients achieve their financial goals through customized solutions and a client-centric approach.\n\n

### Store the Knowledge Base ID / 存储知识库 ID
Store the ID of the generated Unstructured Knowledge Base to use it in the main dual-knowledge-base RAG notebook.

存储生成的非结构化知识库的 ID，以便在主要的双知识库 RAG 笔记本中使用。


In [16]:
kb_region = region
%store unstructured_kb_id
%store kb_region
%store data_bucket_name

unstructured_var = {
    "kb_region": kb_region,
    "unstructured_kb_id": unstructured_kb_id,
    "data_bucket_name": data_bucket_name

}

# Save to current directory
with open("unstructured_var.json", "w") as f:
    json.dump(unstructured_var, f, indent=4)

print("="*60)
print(f"Unstructured Knowledge Base ID: {unstructured_kb_id}")
print(f"Region: {kb_region}")
print(f"S3 Bucket: {data_bucket_name}")

print("="*60)
print("Configuration stored successfully!")



Stored 'unstructured_kb_id' (str)
Stored 'kb_region' (str)
Stored 'data_bucket_name' (str)
Unstructured Knowledge Base ID: P26LUAGL1B
Region: us-east-1
S3 Bucket: octank-financial-unstructured-1938-bucket
Configuration stored successfully!


## Clean up the resources / 清理资源
**If you plan to use the other notebooks and continue with them, do not delete the Knowledge Base yet as it will be needed.**

**如果您计划使用其他笔记本并继续使用它们，请不要删除知识库，因为它将被需要。**

When you are finished with the other notebooks, to avoid additional costs, delete the resources created.

当您完成其他笔记本后，为了避免额外费用，请删除创建的资源。


In [ ]:
# Uncomment to delete resources when finished

# print("===============================Deleting Unstructured Knowledge Base and associated resources==============================\\n")
# unstructured_knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
# print("Cleanup completed successfully!")



###  Summary / 总结
If all the above cells executed successfully, you have:

如果上述所有单元格都成功执行，您已经：

- Created an S3 bucket for unstructured data  
- 为非结构化数据创建了 S3 存储桶
- Uploaded the Octank Financial 10K PDF  
- 上传了 Octank Financial 10K PDF
- Created an Amazon Bedrock Knowledge Base  
- 创建了 Amazon Bedrock 知识库
- Configured OpenSearch Serverless as the vector store  
- 配置了 OpenSearch Serverless 作为向量存储
- Successfully ingested the document  
- 成功摄取了文档
- Tested a query with knowledge base's `RetrieveAndGenerate` API
- 使用知识库的 `RetrieveAndGenerate` API 测试了查询
- Stored the Knowledge Base ID for use in the main notebook  
- 存储了知识库 ID 以供主笔记本使用

You are now ready to proceed to the main `2-unstructured-structured-rag-agent.ipynb` notebook!

您现在可以继续进行主要的 `2-unstructured-structured-rag-agent.ipynb` 笔记本！
